In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch

from pprgo import utils, ppr
from pprgo.pprgo import PPRGo
from pprgo.train import train
from pprgo.predict import predict
from pprgo.dataset import PPRDataset

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

# Download dataset

In [3]:
!wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

--2020-07-16 17:44:26--  https://ndownloader.figshare.com/files/23742119
Resolving proxy.in.tum.de (proxy.in.tum.de)... 131.159.0.2
Connecting to proxy.in.tum.de (proxy.in.tum.de)|131.159.0.2|:8080... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/23742119/reddit.npz [following]
--2020-07-16 17:44:26--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/23742119/reddit.npz
Connecting to proxy.in.tum.de (proxy.in.tum.de)|131.159.0.2|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1480703860 (1,4G) [application/octet-stream]
Saving to: ‘data/reddit.npz’

data/reddit.npz     100%[===================>]   1,38G  73,0MB/s    in 19s     

2020-07-16 17:44:45 (74,2 MB/s) - ‘data/reddit.npz’ saved [1480703860/1480703860]



# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

# Load the data

In [7]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

Runtime: 8.37s


# Preprocessing: Calculate PPR scores

In [8]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR
start = time.time()
topk_train = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk,
                                 normalization=ppr_normalization)
train_set = PPRDataset(attr_matrix_all=attr_matrix, ppr_matrix=topk_train, indices=train_idx, labels_all=labels)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
    val_set = PPRDataset(attr_matrix_all=attr_matrix, ppr_matrix=topk_val, indices=val_idx, labels_all=labels)
else:
    val_set = None
time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")

Runtime: 0.92s


# Training: Set up model and train

In [9]:
start = time.time()
model = PPRGo(d, nc, hidden_size, nlayers, dropout)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

nepochs, _, _ = train(
        model=model, train_set=train_set, val_set=val_set,
        lr=lr, weight_decay=weight_decay,
        max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
        eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2020-07-16 17:45:00 (INFO): Epoch 9, step 20: train 0.00916
2020-07-16 17:45:00 (INFO): Epoch 19, step 40: train 0.00872
2020-07-16 17:45:00 (INFO): Epoch 29, step 60: train 0.00842
2020-07-16 17:45:00 (INFO): Epoch 39, step 80: train 0.00821
2020-07-16 17:45:00 (INFO): Epoch 49, step 100: train 0.00803
2020-07-16 17:45:00 (INFO): Epoch 59, step 120: train 0.00787
2020-07-16 17:45:00 (INFO): Epoch 69, step 140: train 0.00773
2020-07-16 17:45:00 (INFO): Epoch 79, step 160: train 0.00759
2020-07-16 17:45:01 (INFO): Epoch 89, step 180: train 0.00746
2020-07-16 17:45:01 (INFO): Epoch 99, step 200: train 0.00734
2020-07-16 17:45:01 (INFO): Epoch 109, step 220: train 0.00723
2020-07-16 17:45:01 (INFO): Epoch 119, step 240: train 0.00712
2020-07-16 17:45:01 (INFO): Epoch 129, step 260: train 0.00701
2020-07-16 17:45:01 (INFO): Epoch 139, step 280: train 0.00691
2020-07-16 17:45:01 (INFO): Epoch 149, step 300: train 0.00682
2020-07-16 17:45:01 (INFO): Epoch 159, step 320: train 0.00673
2020-07

Runtime: 2.87s


# Inference (val and test)

In [10]:
start = time.time()
predictions, time_logits, time_propagation = predict(
        model=model, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 8.00s


# Collect and print results

In [11]:
acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])
f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')

gpu_memory = torch.cuda.max_memory_allocated()
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [12]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB
''')


Accuracy: Train: 37.2%, val: 27.0%, test: 27.2%
F1 score: Train: 0.188, val: 0.112, test: 0.112

Runtime: Preprocessing: 0.92s, training: 2.87s, inference: 8.00s -> total: 11.78s
Memory: Main: 5.38GB, GPU: 0.045GB

